### This code creates the results used in Section 6.3

In [3]:
import numpy as np
from tigramite.pcmci import PCMCI
from tigramite.independence_tests.parcorr import ParCorr
import tigramite.data_processing as pp
import random
import pickle
from scipy.stats import binomtest


In [5]:
# The function that generates the dynamical data

def lordata_gen(size):
    X=np.zeros((size+1,6))
    X[0,:]=[2,0.97,0.99,1,0.97,1]
    
    
    for t in range(size):
        e=np.random.multivariate_normal(np.zeros(6),np.eye(6))
        x_new=[X[t,0]*0.9+0.1*X[t,1]+e[0],\
               0.28*X[t,0]-0.01*X[t,0]*X[t,2]+0.99*X[t,1]+e[1],\
               0.01*X[t,0]*X[t,1]-0.01*X[t,0]*X[t,3]+0.9733*X[t,2]+e[2],\
               0.01*X[t,0]*X[t,2]-0.02*X[t,0]*X[t,4]+0.9366*X[t,3]+e[3],\
               0.02*X[t,0]*X[t,3]+0.96*X[t,4]+e[4],
                  X[t,5]+e[5]]
        X[t+1,:]=x_new

    return(X)


In [20]:
# Running PCMCI, with partial correlation
np.random.seed(0)
reportedG=np.zeros((6,6))
E=300
runs=500
size=8500
n=25
for j in range(runs):
    X=lordata_gen(size)
    print('Run',j,'out of',runs)
    for e in range(E):
        if e%10==0:
            data=X[499+e*n:499+(e+1)*n,:]
            dataframe = pp.DataFrame(data)
            cond_ind_test = ParCorr()
            pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cond_ind_test)
            results = pcmci.run_pcmci(tau_min=1,tau_max=1, pc_alpha=None)
            reportedG+=(results['p_matrix'][:,:,1]<0.1)
            print(results['p_matrix'][:,:,1]<0.1)

Run 0 out of 500
Run 1 out of 500
Run 2 out of 500
Run 3 out of 500
Run 4 out of 500
Run 5 out of 500
Run 6 out of 500
Run 7 out of 500
Run 8 out of 500
Run 9 out of 500
Run 10 out of 500
Run 11 out of 500
Run 12 out of 500
Run 13 out of 500
Run 14 out of 500
Run 15 out of 500
Run 16 out of 500
Run 17 out of 500
Run 18 out of 500
Run 19 out of 500
Run 20 out of 500
Run 21 out of 500
Run 22 out of 500
Run 23 out of 500
Run 24 out of 500
Run 25 out of 500
Run 26 out of 500
Run 27 out of 500
Run 28 out of 500
Run 29 out of 500
Run 30 out of 500
Run 31 out of 500
Run 32 out of 500
Run 33 out of 500
Run 34 out of 500
Run 35 out of 500
Run 36 out of 500
Run 37 out of 500
Run 38 out of 500
Run 39 out of 500
Run 40 out of 500
Run 41 out of 500
Run 42 out of 500
Run 43 out of 500
Run 44 out of 500
Run 45 out of 500
Run 46 out of 500
Run 47 out of 500
Run 48 out of 500
Run 49 out of 500
Run 50 out of 500
Run 51 out of 500
Run 52 out of 500
Run 53 out of 500
Run 54 out of 500
Run 55 out of 500
Ru

In [54]:
# The final counts
print(reportedG)

[[10778.  1906.  2209.  1996.  2033.  1868.]
 [ 2422. 12439.  2035.  1835.  1830.  1870.]
 [ 1766.  2095. 12816.  2239.  1852.  1809.]
 [ 1875.  1816.  2201. 12874.  3273.  1838.]
 [ 1909.  1848.  2000.  3030. 12980.  1960.]
 [ 1857.  1832.  1851.  1898.  1879.  9968.]]


In [45]:
Ground

array([[1, 1, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 0, 0],
       [1, 0, 1, 1, 1, 0],
       [1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 1]])

In [22]:
# If we find edges from PCMCI with the same thresholding we receive a fully connected graph

    
final=reportedG
pvals=np.zeros((6,6))
for i in range(6):
    for j in range(6):
        test=binomtest(np.int32(reportedG[i,j]),500*30,0.1,alternative='greater')
        pvals[i,j]=test.pvalue
print('Maxtrix of causal parents for n=',n)
print(pvals<0.05)

Maxtrix of causal parents for n= 25
[[ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]]


In [55]:
# So we instead chose an optimal threshold to report causal connections

Ground=np.array([[1,1,1,1,1,0],[1,1,1,0,0,0],[0,1,1,1,0,0],[0,0,1,1,1,0],[0,0,0,1,1,0],[0,0,0,0,0,1]]) #Grundtruth relations
thresh=reportedG>1994 # Thresh is the optimal graph
print(Ground-thresh) # Only two errors in the difference
print(thresh)

[[ 0  1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0 -1  0  0  0]
 [ 0  0  0  0  0  0]]
[[ True False  True  True  True False]
 [ True  True  True False False False]
 [False  True  True  True False False]
 [False False  True  True  True False]
 [False False  True  True  True False]
 [False False False False False  True]]
